In [ ]:
# © 2022. Triad National Security, LLC. All rights reserved.
# This program was produced under U.S. Government contract 89233218CNA000001 for Los Alamos
# National Laboratory (LANL), which is operated by Triad National Security, LLC for the U.S.
# Department of Energy/National Nuclear Security Administration. All rights in the program are
# reserved by Triad National Security, LLC, and the U.S. Department of Energy/National Nuclear
# Security Administration. The Government is granted for itself and others acting on its behalf a
# nonexclusive, paid-up, irrevocable worldwide license in this material to reproduce, prepare
# derivative works, distribute copies to the public, perform publicly and display publicly, and to permit
# others to do so.

In [ ]:
import os
import time
import joblib
import h5py
import random as python_random

import tensorflow as tf
import numpy as np
import scipy as sp

gpus = tf.config.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)
print(len(gpus), "Physical GPUs")

import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams.update({'font.size': 16})
mpl.rcParams.update({'font.family': 'serif'})

from utils_data_windows import write_tfrecord_train_valid_data, write_tfrecord_test_data, read_tfrecord_dataset
from utils_data_windows import write_tfrecord_train_valid_data_major_slips, get_mu_slip_loc_data_with_index
from utils_labquake_model import LabquakeModel, LabquakeModel_MS, MuModel
from utils_model_training import train_labquake_model, get_labquake_model
from utils_model_training import test_labquake_model, test_labquake_model_detail

from sklearn.metrics import r2_score

seed_value = 1234
python_random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

In [ ]:
verbose = 1

# Parameters data dict
params_data = {}
params_data['filenames_train']    = ['./Data/p4677_seismic_data_2MPa.hdf5', './Data/p4677_shear_data_2MPa.hdf5']
params_data['train_data_split']   = 0.6
params_data['valid_data_split']   = 0.4

params_data['in_scaler_mode']     = 'standard'
params_data['out_scaler_mode']    = 'minmax'
params_data['sample_freq']        = 1000.0
params_data['unit_time_intvl']    = 0.008
params_data['num_in_time_intvl']  = 32
params_data['num_out_time_intvl'] = 1
params_data['in_time_intvl']      = params_data['unit_time_intvl'] * params_data['num_in_time_intvl']
params_data['out_time_intvl']     = params_data['unit_time_intvl'] * params_data['num_out_time_intvl']
params_data['in_out_time_diff']   = params_data['in_time_intvl']
params_data['time_advance']       = params_data['out_time_intvl']

params_data['batch_size']         = 64 * 8
params_data['shuffle_buffer']     = int(1e4)

# tfrecord dataset
DATA_DIR = './'
params_data['tfrecord_window_data'] = DATA_DIR+'tfrecord_window_'+str(params_data['in_time_intvl'])+'_'+str(params_data['out_time_intvl'])+'/'
params_data['num_unit_ae']          = 512

# Major slip positions
p4677_train_fail_index = np.load('./Data/p4677_train_fail_index.npy')
p4677_valid_fail_index = np.load('./Data/p4677_valid_fail_index.npy')
p4581_test_fail_index = np.load('./Data/p4581_3mpa_stepup_fail_index.npy')
params_data['slip_seg_size'] = 4

In [ ]:
# Load Train/Valid datasets
train_dataset = read_tfrecord_dataset(params_data, 'train_data.tfrecord', is_shuffle=True)
valid_dataset = read_tfrecord_dataset(params_data, 'valid_data.tfrecord', is_shuffle=False)
options = tf.data.Options()
options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.DATA
train_dataset = train_dataset.with_options(options)
valid_dataset = valid_dataset.with_options(options)

train_data_size = np.load(params_data['tfrecord_window_data']+'train_dataset_size.npy')[0]

# Load Train/Valid major slips datasets
train_major_slips_dataset = read_tfrecord_dataset(params_data, 'train_major_slips_data.tfrecord', is_shuffle=True)
valid_major_slips_dataset = read_tfrecord_dataset(params_data, 'valid_major_slips_data.tfrecord', is_shuffle=False)
options = tf.data.Options()
options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.DATA
train_major_slips_dataset = train_major_slips_dataset.with_options(options)
valid_major_slips_dataset = valid_major_slips_dataset.with_options(options)

train_major_slips_data_size = np.load(params_data['tfrecord_window_data']+'train_major_slips_dataset_size.npy')[0]

# Load Train Scaler
input_scaler_train = joblib.load(params_data['tfrecord_window_data']+'input_scaler_train.save')
output_scaler_train = joblib.load(params_data['tfrecord_window_data']+'output_scaler_train.save')
print(input_scaler_train.mean_,np.sqrt(input_scaler_train.var_))
print(output_scaler_train.data_min_, output_scaler_train.data_max_)
print('')

# Load Test Scaler
input_scaler_test = joblib.load(params_data['tfrecord_window_data']+'input_scaler_test.save')
output_scaler_test = joblib.load(params_data['tfrecord_window_data']+'output_scaler_test.save')
print(input_scaler_test.mean_,np.sqrt(input_scaler_test.var_))
print(output_scaler_test.data_min_, output_scaler_test.data_max_)

In [ ]:
# Parameters model
params_model = {}

params_model['num_vq_codes']        = 24 + 8
params_model['embedding_dim']       = int(params_data['unit_time_intvl']*params_data['sample_freq'])
params_model['commitment_cost']     = 0.25
params_model['ema_decay']           = 0.99

params_model['save_model_file']    = './trsf_future_model-'+str(params_data['in_time_intvl'])+'-'+str(params_data['out_time_intvl'])
params_model['transformer_max_len'] = 1000
params_model['num_train_epochs']    = 500
params_model['learning_rate']       = 1e-4
params_model['lr_warmup_epochs']    = 10
params_model['lr_decay_epochs']     = 100
params_model['steps_per_epoch']     = int(np.ceil(train_data_size/params_data['batch_size']))
params_model['min_delta_stop']      = 1e-4
params_model['early_stop_wait']     = 50
params_model['verbose_model_train'] = 1

# Load Mu VQ model
num_unit = int(params_data['sample_freq']*params_data['unit_time_intvl'])
mu_dataset = tf.data.Dataset.from_tensor_slices((tf.zeros((1,num_unit,1)), tf.zeros((1,num_unit,1))))
mu_dataset = mu_dataset.batch(1)
mu_model = get_labquake_model(params_data, params_model, 
                              MuModel, mu_dataset, pre_weight='./mu_vq_latent_model.h5')
mu_embeddings = mu_model.vq_layer.embeddings.numpy()
params_model['mu_embeddings'] = mu_embeddings

In [ ]:
# Init model
labquake_model = LabquakeModel_MS(params_data, params_model)
for ds in train_major_slips_dataset.take(1):
  x, y = labquake_model.get_in_out_data(ds)
_ = labquake_model(x, training=False)

# Load Step2 model
labquake_model.load_weights(params_model['save_model_file']+'/model_weights.h5')
labquake_model.ae_encoder.trainable = False
for i in range(labquake_model.num_layers):
  labquake_model.trsf_enc_layers[i].trainable = False
  labquake_model.trsf_dec_layers[i].trainable = False
for ii in range(len(labquake_model.mu_decoder)):
  labquake_model.mu_decoder[ii].trainable = False
_ = labquake_model.call_slip(x, training=False)
for ii in range(len(labquake_model.slip_decoder)):
  labquake_model.slip_decoder[ii].trainable = True

labquake_model.summary()

In [ ]:
# Optimizer
optimizer = tf.keras.optimizers.Adam(params_model['learning_rate'])

# Earlystop callback
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=params_model['min_delta_stop'], patience=params_model['early_stop_wait'], 
                                              restore_best_weights=True, verbose=params_model['verbose_model_train'], mode='min')
callbacks = [early_stop]

# Create model
labquake_model.compile(optimizer=optimizer)

# Training
start_time = time.time()
if params_model['verbose_model_train']:
    print('\n Model Training')

history = labquake_model.fit(train_major_slips_dataset, 
                           epochs=params_model['num_train_epochs'], 
                           validation_data=valid_major_slips_dataset, 
                           callbacks=callbacks, 
                           verbose=params_model['verbose_model_train'])

# Evaluate
train_results = labquake_model.evaluate(train_major_slips_dataset, verbose=params_model['verbose_model_train'])
valid_results = labquake_model.evaluate(valid_major_slips_dataset, verbose=params_model['verbose_model_train'])

end_time = time.time()
if params_model['verbose_model_train']:
    print('\n Model Training Time', end_time-start_time, 's')

# Save model
tf.saved_model.save(labquake_model, params_model['save_model_file'])

# Train History
plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'], 'r-', label='train')
plt.plot(history.history['val_loss'], 'b--', label='valid')
plt.legend()

In [ ]:
# Load model
labquake_model = tf.saved_model.load(params_model['save_model_file'])

# Valid data
test_dataset = read_tfrecord_dataset(params_data, 'valid_data.tfrecord', is_shuffle=False)
input_scaler = input_scaler_train
output_scaler = output_scaler_train
fail_index = p4677_valid_fail_index

# Predict
targets, preds, targets_time, preds_slip, inputs, inputs_time, attn = test_labquake_model_detail(labquake_model, test_dataset, input_scaler, output_scaler)
mu_slip_loc_test, mu_slip_index = get_mu_slip_loc_data_with_index(params_data, targets, fail_index)
target_signal = np.concatenate(targets, axis=0)
predict_signal = np.concatenate(preds, axis=0)
target_signal_time = np.concatenate(targets_time, axis=0)
R2_Curve = r2_score(target_signal, predict_signal)
MAPE_Curve = tf.keras.metrics.mean_absolute_percentage_error(target_signal[:,0], predict_signal[:,0])/100.0
MSE_Curve = tf.keras.metrics.mean_squared_error(target_signal[:,0], predict_signal[:,0])
print('  Full Signal R2: {:1.3}'.format(R2_Curve))
print('  Full Signal MAPE: {:1.3%}'.format(MAPE_Curve))
print('  Full Signal MSE: {:1.3}'.format(MSE_Curve))

# Plot
fig = plt.figure(figsize=(20, 10))
ax = plt.gca()
ax.plot(target_signal_time, target_signal, 'b-', linewidth=2)
ax.plot(target_signal_time, predict_signal, 'r-', linewidth=2, alpha=1)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

fig = plt.figure(figsize=(20, 25))
outer = gridspec.GridSpec(4, 5, wspace=0.4, hspace=0.4)

attn_score = np.mean(attn[:,:,0,:], axis=1)
attn_score = np.repeat(attn_score, int(params_data['num_in_time_intvl']*8/attn_score.shape[1]), axis=-1)
in_time = (0.0 - np.arange(int(params_data['num_in_time_intvl']*8),0,-1.0))/1000.0
out_time = 0.0 + np.arange(0,int(params_data['num_out_time_intvl']*8),1)/1000.0

for i in range(len(mu_slip_index)):
    inner = gridspec.GridSpecFromSubplotSpec(2, 1,
                    subplot_spec=outer[i], wspace=0.0, hspace=0.2)

    ax1 = plt.Subplot(fig, inner[0])
    ax1.plot(in_time, inputs[mu_slip_index[i],:,:]/1000.0, 'k-', linewidth=2.0)
    ax1.set_xlim([np.min(in_time), 0.0])
    ax1.set_ylim([0.0, 2.0])
    if i == 0:
        ax1.set_ylabel('Input scaled |AE|', color='k')
        ax1.set_yticks([0.0, 1.0, 2.0])
    else:
        ax1.set_yticks([])
        ax1.set_xticks([])
    fig.add_subplot(ax1)
    
    ax2 = ax1.twinx()
    ax2.plot(in_time, attn_score[mu_slip_index[i],:], 'm-', linewidth=2.0, alpha=0.75)
    ax2.tick_params(axis='y', labelcolor='m')
    ax1.set_xlim([np.min(in_time), 0.0])
    ax2.set_ylim([0.0, 0.2])
    if i == 0:
        ax2.set_ylabel('Output $\mu$', color='r')
        ax2.set_yticks([0.0, 0.1, 0.2])
        ax2.set_ylabel('Attention', color='m')
    else:
        ax2.set_yticks([])
        ax2.set_xticks([])
    fig.add_subplot(ax2)

    ax3 = plt.Subplot(fig, inner[1])
    ax3.plot(out_time, preds_slip[mu_slip_index[i],:,:], 'r--', linewidth=2, alpha=1)
    ax3.plot(out_time, targets[mu_slip_index[i],:,:], 'k-', linewidth=1.5, alpha=1)
    ax3.tick_params(axis='y', labelcolor='r')
    ax3.set_ylim([0.4, 0.7])
    ax3.set_xlim([0.0, np.max(out_time)+0.001])
    if i == 0:
        ax3.set_ylabel('Output $\mu$', color='r')
        ax3.set_yticks([0.45, 0.55, 0.65])
        ax3.set_xlabel('Time [s]')
    else:
        ax3.set_yticks([])
        ax3.set_xticks([])
    fig.add_subplot(ax3)

fig.show()

In [ ]:
# Test data
test_dataset = read_tfrecord_dataset(params_data, 'test_data.tfrecord', is_shuffle=False)
input_scaler = input_scaler_test
output_scaler = output_scaler_test
fail_index = p4581_test_fail_index

# Predict
targets, preds, targets_time, preds_slip, inputs, inputs_time, attn = test_labquake_model_detail(labquake_model, test_dataset, input_scaler, output_scaler)
mu_slip_loc_test, mu_slip_index = get_mu_slip_loc_data_with_index(params_data, targets, fail_index)
target_signal = np.concatenate(targets, axis=0)
predict_signal = np.concatenate(preds, axis=0)
target_signal_time = np.concatenate(targets_time, axis=0)
R2_Curve = r2_score(target_signal, predict_signal)
MAPE_Curve = tf.keras.metrics.mean_absolute_percentage_error(target_signal[:,0], predict_signal[:,0])/100.0
MSE_Curve = tf.keras.metrics.mean_squared_error(target_signal[:,0], predict_signal[:,0])
print('  Full Signal R2: {:1.3}'.format(R2_Curve))
print('  Full Signal MAPE: {:1.3%}'.format(MAPE_Curve))
print('  Full Signal MSE: {:1.3}'.format(MSE_Curve))

# Plot
fig = plt.figure(figsize=(20, 10))
ax = plt.gca()
ax.plot(target_signal_time, target_signal, 'b-', linewidth=2)
ax.plot(target_signal_time, predict_signal, 'r-', linewidth=2, alpha=1)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

attn_score = np.mean(attn[:,:,0,:], axis=1)
attn_score = np.repeat(attn_score, int(params_data['num_in_time_intvl']*8/attn_score.shape[1]), axis=-1)
in_time = (0.0 - np.arange(int(params_data['num_in_time_intvl']*8),0,-1.0))/1000.0
out_time = 0.0 + np.arange(0,int(params_data['num_out_time_intvl']*8),1)/1000.0

fig = plt.figure(figsize=(20, 25))
outer = gridspec.GridSpec(4, 5, wspace=0.4, hspace=0.4)

for i in range(len(mu_slip_index)):
    inner = gridspec.GridSpecFromSubplotSpec(2, 1,
                    subplot_spec=outer[i], wspace=0.0, hspace=0.2)

    ax1 = plt.Subplot(fig, inner[0])
    ax1.plot(in_time, inputs[mu_slip_index[i],:,:]/1000.0, 'k-', linewidth=2.0)
    ax1.set_xlim([np.min(in_time), 0.0])
    ax1.set_ylim([0.0, 2.0])
    if i == 0:
        ax1.set_ylabel('Input scaled |AE|', color='k')
        ax1.set_yticks([0.0, 1.0, 2.0])
    else:
        ax1.set_yticks([])
        ax1.set_xticks([])
    fig.add_subplot(ax1)
    
    ax2 = ax1.twinx()
    ax2.plot(in_time, attn_score[mu_slip_index[i],:], 'm-', linewidth=2.0, alpha=0.75)
    ax2.tick_params(axis='y', labelcolor='m')
    ax1.set_xlim([np.min(in_time), 0.0])
    ax2.set_ylim([0.0, 0.2])
    if i == 0:
        ax2.set_ylabel('Output $\mu$', color='r')
        ax2.set_yticks([0.0, 0.1, 0.2])
        ax2.set_ylabel('Attention', color='m')
    else:
        ax2.set_yticks([])
        ax2.set_xticks([])
    fig.add_subplot(ax2)

    ax3 = plt.Subplot(fig, inner[1])
    ax3.plot(out_time, preds_slip[mu_slip_index[i],:,:], 'r--', linewidth=2, alpha=1)
    ax3.plot(out_time, targets[mu_slip_index[i],:,:], 'k-', linewidth=1.5, alpha=1)
    ax3.tick_params(axis='y', labelcolor='r')
    ax3.set_ylim([0.31, 0.47])
    ax3.set_xlim([0.0, np.max(out_time)+0.001])
    if i == 0:
        ax3.set_ylabel('Output $\mu$', color='r')
        ax3.set_yticks([0.35, 0.45])
        ax3.set_xlabel('Time [s]')
    else:
        ax3.set_yticks([])
        ax3.set_xticks([])
    fig.add_subplot(ax3)

fig.show()